In [ ]:
import pandas as pd
import yfinance as yf
from datetime import datetime, timedelta
import pandas_ta as ta
import numpy as np
import requests
from bs4 import BeautifulSoup

# Function to fetch S&P 500 tickers from Wikipedia
# def get_sp500_tickers():
#     url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
#     response = requests.get(url)
#     soup = BeautifulSoup(response.text, 'html.parser')
#     table = soup.find('table', {'id': 'constituents'})
#     tickers = [row.find('td').text.strip() for row in table.find_all('tr')[1:]]
#     return tickers

# # Get the S&P 500 tickers
# sp500_tickers = get_sp500_tickers()

# # Save the tickers to a CSV file
# pd.DataFrame(sp500_tickers, columns=["Ticker"]).to_csv('sp500_tickers.csv', index=False)


In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

# Function to fetch S&P 500 tickers from Wikipedia
def get_sp500_tickers():
    url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    table = soup.find('table', {'id': 'constituents'})
    tickers = [row.find('td').text.strip() for row in table.find_all('tr')[1:]]
    return tickers# Get only the first 10 tickers

# Get the S&P 500 tickers
sp500_tickers = get_sp500_tickers()

# Display the tickers
print("S&P 500 Tickers:")
print(sp500_tickers)

# Save the tickers to a CSV file
pd.DataFrame(sp500_tickers, columns=["Ticker"]).to_csv('sp500_tickers.csv', index=False)


In [ ]:

# Define the timeframes and their corresponding date ranges

# Read the tickers from the CSV file
tickers_df = pd.read_csv('sp500_tickers.csv')
tickers = tickers_df['Ticker'].tolist()

timeframes = {
    # '15m': timedelta(days=60),
    # '1h': timedelta(days=730),
    '1d': timedelta(days=180),
    # '1wk': timedelta(days=3*365)
}

# Define the recent period (in days)
recent_period = 1

# Function to download data for a given ticker and timeframe
def download_data(ticker, timeframe, start_date, end_date):
    return yf.download(ticker, start=start_date, end=end_date, interval=timeframe)

# Dictionary to store data for each timeframe
data = {tf: {} for tf in timeframes}

# Download data for each ticker and timeframe
end_date = datetime.now()
for ticker in tickers:
    for tf, delta in timeframes.items():
        start_date = end_date - delta
        try:
            data[tf][ticker] = download_data(ticker, tf, start_date, end_date)
        except Exception as e:
            print(f"Failed to download data for {ticker} with timeframe {tf}: {e}")


In [ ]:


# List to store screener results
screener_results = []

# Applying the custom indicator and generating buy/sell signals
for tf in timeframes:
    for ticker in data[tf]:
        df = data[tf][ticker]
        
        # Round OHLC values to two decimal points
        df['Open'] = df['Open'].round(2)
        df['High'] = df['High'].round(2)
        df['Low'] = df['Low'].round(2)
        df['Close'] = df['Close'].round(2)
        
       # --- Existing calculations ---
        # Linear Regression Curves
        df['reg1'] = ta.linreg(df['Close'], length=10)
        df['reg2'] = ta.linreg(df['Close'], length=14)
        df['reg3'] = ta.linreg(df['Close'], length=30)

        # R-squared Calculation
        r2_length = 14
        df['r2_raw'] = df['Close'].rolling(window=r2_length).apply(
            lambda x: np.corrcoef(x, np.arange(r2_length))[0, 1]**2
        )
        df['r2'] = df['r2_raw'] * 100  # Normalized to [0, 100]
        df['r2_smoothed'] = df['r2'].rolling(window=3).mean()

        # --- Flatten the columns if they are a MultiIndex ---
        if isinstance(df.columns, pd.MultiIndex):
            # This will take the first level of the MultiIndex as the new columns.
            df.columns = df.columns.get_level_values(0)

        # --- RSI Calculation using the DataFrame accessor ---
        # Explicitly specify the 'Close' column.
        df.ta.rsi(close='Close', length=14, append=True)
        # This will add a new column named 'RSI_14' to df.

        # --- Buy and Sell Signals ---
        df['buy_signal'] = np.where((df['r2_smoothed'] > 90) & (df['RSI_14'] < 30), 1, 0)
        df['sell_signal'] = np.where((df['r2_smoothed'] > 90) & (df['RSI_14'] > 70), 1, 0)


                
                # Save the DataFrame to a CSV file
        df.to_csv(f'{ticker}_{tf}_data.csv')
        
        
        # Filter for recent periods
        recent_date_cutoff = df.index.max() - pd.Timedelta(days=recent_period)
        df = df[df.index >= recent_date_cutoff]
        
        # Filter rows with buy or sell signals
        df_filtered = df[(df['buy_signal'] == 1) | (df['sell_signal'] == 1)]
        
        # Append results to screener list
        if not df_filtered.empty:
            for index, row in df_filtered.iterrows():
                screener_results.append({
                    'Ticker': ticker,
                    'Date': index,
                    'Buy Signal': row['buy_signal'],
                    'Sell Signal': row['sell_signal']
                })

# Save screener results to a CSV file
screener_df = pd.DataFrame(screener_results)
# screener_df.to_csv('screener_results.csv', index=False)

# Display a sample of the screener results
print("Screener results:")
print(screener_df)


# regession line crossed ta alerts

In [ ]:

# Read the tickers from the CSV file
tickers_df = pd.read_csv('sp500_tickers.csv')
tickers = tickers_df['Ticker'].tolist()

# Define the timeframes and their corresponding date ranges
timeframes = {
    # '15m': timedelta(days=60),
    '1h': timedelta(days=30),
    # '1d': timedelta(days=3*365),
    # '1wk': timedelta(days=3*365)
}

# Define the recent period (in days)
recent_period = 1

# Function to download data for a given ticker and timeframe
def download_data(ticker, timeframe, start_date, end_date):
    return yf.download(ticker, start=start_date, end=end_date, interval=timeframe)

# Dictionary to store data for each timeframe
data = {tf: {} for tf in timeframes}

# Download data for each ticker and timeframe
end_date = datetime.now()
for ticker in tickers:
    for tf, delta in timeframes.items():
        start_date = end_date - delta
        try:
            data[tf][ticker] = download_data(ticker, tf, start_date, end_date)
        except Exception as e:
            print(f"Failed to download data for {ticker} with timeframe {tf}: {e}")


In [ ]:

# List to store screener results
screener_results = []

# Applying the custom indicator and generating buy/sell signals
for tf in timeframes:
    for ticker in data[tf]:
        df = data[tf][ticker]
        
                # If the DataFrame columns are a MultiIndex, flatten them
        if isinstance(df.columns, pd.MultiIndex):
            df.columns = df.columns.get_level_values(0)

        
        # Round OHLC values to two decimal points
        df['Open'] = df['Open'].round(2)
        df['High'] = df['High'].round(2)
        df['Low'] = df['Low'].round(2)
        df['Close'] = df['Close'].round(2)
        
       # --- Existing calculations ---
        # Linear Regression Curves
        
        df['reg1'] = ta.linreg(df['Close'], length=25)
        df['reg2'] = ta.linreg(df['Close'], length=50)
        
        
        # df['reg3'] = ta.linreg(df['Close'], length=30)

        # R-squared Calculation
        # r2_length = 14
        # df['r2_raw'] = df['Close'].rolling(window=r2_length).apply(
        #     lambda x: np.corrcoef(x, np.arange(r2_length))[0, 1]**2
        # )
        # df['r2'] = df['r2_raw'] * 100  # Normalized to [0, 100]
        # df['r2_smoothed'] = df['r2'].rolling(window=3).mean()

        # --- Flatten the columns if they are a MultiIndex ---
        # if isinstance(df.columns, pd.MultiIndex):
        #     # This will take the first level of the MultiIndex as the new columns.
        #     df.columns = df.columns.get_level_values(0)

        # --- RSI Calculation using the DataFrame accessor ---
        # Explicitly specify the 'Close' column.
        # df.ta.rsi(close='Close', length=14, append=True)
        # This will add a new column named 'RSI_14' to df.

        # --- Buy and Sell Signals ---
        # df['buy_signal'] = np.where((df['r2_smoothed'] > 90) & (df['RSI_14'] < 30), 1, 0)
        # df['sell_signal'] = np.where((df['r2_smoothed'] > 90) & (df['RSI_14'] > 70), 1, 0)

        df['buy_signal'] = np.where(
            (df['reg1'] > df['reg2']) & (df['reg1'].shift(1) <= df['reg2'].shift(1)),
            1,
            0
        )
        
        # Sell Signal: When the 25-period line (reg1) crosses below the 50-period line (reg2)
        df['sell_signal'] = np.where(
            (df['reg1'] < df['reg2']) & (df['reg1'].shift(1) >= df['reg2'].shift(1)),
            1,
            0
        )
                
                # Save the DataFrame to a CSV file
        df.to_csv(f'{ticker}_{tf}_data.csv')
        
        
        # Filter for recent periods
        recent_date_cutoff = df.index.max() - pd.Timedelta(days=recent_period)
        df = df[df.index >= recent_date_cutoff]
        
        # Filter rows with buy or sell signals
        df_filtered = df[(df['buy_signal'] == 1) | (df['sell_signal'] == 1)]
        
        # Append results to screener list
        if not df_filtered.empty:
            for index, row in df_filtered.iterrows():
                screener_results.append({
                    'Ticker': ticker,
                    'Date': index,
                    'Buy Signal': row['buy_signal'],
                    'Sell Signal': row['sell_signal']
                })

# Save screener results to a CSV file
screener_df = pd.DataFrame(screener_results)
# screener_df.to_csv('screener_results.csv', index=False)

# Display a sample of the screener results
print("Screener results:")
print(screener_df)


# test previous code

In [ ]:
import pytz
from datetime import datetime, timedelta




    
import pandas as pd
import yfinance as yf
from datetime import datetime, timedelta
import pandas_ta as ta
import numpy as np
import requests
from bs4 import BeautifulSoup
import os
from dotenv import load_dotenv
load_dotenv()


# Function to fetch S&P 500 tickers from Wikipedia
def get_sp500_tickers():
    url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    table = soup.find('table', {'id': 'constituents'})
    tickers = [row.find('td').text.strip() for row in table.find_all('tr')[1:]]
    return tickers

# Function to send message to Telegram
def send_telegram_message(message):
    bot_token = os.getenv("TELEGRAM_BOT_TOKEN")
    chat_id = os.getenv("TELEGRAM_CHAT_ID")
    url = f"https://api.telegram.org/bot{bot_token}/sendMessage"
    data = {"chat_id": chat_id, "text": message}
    requests.post(url, data=data)

    # Get the S&P 500 tickers
sp500_tickers = get_sp500_tickers()

# Ensure the stockdata directory exists
os.makedirs('stockdata', exist_ok=True)

# Save the tickers to a CSV file in the stockdata folder
pd.DataFrame(sp500_tickers, columns=["Ticker"]).to_csv('stockdata/sp500_tickers.csv', index=False)

# Read the tickers from the CSV file in the stockdata folder
tickers_df = pd.read_csv('stockdata/sp500_tickers.csv')
tickers = tickers_df['Ticker'].tolist()

# Define the timeframes and their corresponding date ranges
timeframes = {
    # '15m': timedelta(days=60),
    '1h': timedelta(days=30),
    # '1d': timedelta(days=90),
    # '1wk': timedelta(days=3*365)
}

# Define the recent period (in days)
recent_period = 1

# Function to download data for a given ticker and timeframe
def download_data(ticker, timeframe, start_date, end_date):
    return yf.download(ticker, start=start_date, end=end_date, interval=timeframe)

# Dictionary to store data for each timeframe
data = {tf: {} for tf in timeframes}

# Download historical data for each ticker and timeframe
end_date = datetime.now()
for ticker in tickers:
    for tf, delta in timeframes.items():
        start_date = end_date - delta
        try:
            df = download_data(ticker, tf, start_date, end_date)
            if df.empty:
                print(f"No data available for {ticker} on timeframe {tf}. Skipping.")
                continue
            data[tf][ticker] = df
        except Exception as e:
            print(f"Failed to download data for {ticker} with timeframe {tf}: {e}")



# List to store screener results
screener_results = []

# Applying the custom indicator and generating buy/sell signals
for tf in timeframes:
    for ticker in data[tf]:
        df = data[tf][ticker]
        
                # If the DataFrame columns are a MultiIndex, flatten them
        if isinstance(df.columns, pd.MultiIndex):
            df.columns = df.columns.get_level_values(0)

        
        # Round OHLC values to two decimal points
        df['Open'] = df['Open'].round(2)
        df['High'] = df['High'].round(2)
        df['Low'] = df['Low'].round(2)
        df['Close'] = df['Close'].round(2)
        
    # --- Existing calculations ---
        # Linear Regression Curves
        
        df['reg1'] = ta.linreg(df['Close'], length=25)
        df['reg2'] = ta.linreg(df['Close'], length=50)
        

        df['buy_signal'] = np.where(
            (df['reg1'] > df['reg2']) & (df['reg1'].shift(1) <= df['reg2'].shift(1)),
            1,
            0
        )
        
        # Sell Signal: When the 25-period line (reg1) crosses below the 50-period line (reg2)
        df['sell_signal'] = np.where(
            (df['reg1'] < df['reg2']) & (df['reg1'].shift(1) >= df['reg2'].shift(1)),
            1,
            0
        )
                
        # Save each ticker's data to a CSV file in the stockdata folder
        df.to_csv(f'stockdata/{ticker}_{tf}_data.csv')

        
        hoursback =1 
        # Filter for recent periods
        recent_date_cutoff = df.index.max() - pd.Timedelta(hours=hoursback)
        df = df[df.index >= recent_date_cutoff]
        
        # Filter rows with buy or sell signals
        df_filtered = df[(df['buy_signal'] == 1) | (df['sell_signal'] == 1)]
        
        # Append results to screener list
        if not df_filtered.empty:
            for index, row in df_filtered.iterrows():
                screener_results.append({
                    'Ticker': ticker,
                    'Date': index,
                    'Buy Signal': row['buy_signal'],
                    'Sell Signal': row['sell_signal']
                })

# Save screener results to a CSV file
screener_df = pd.DataFrame(screener_results)
screener_df.to_csv('Regression_cross_screener_results_1h.csv', index=False)


# Send results to Telegram
if not screener_df.empty:
    message = "Hourly screener Results:\n Buy=linreg 25 crossover linreg 50 \n Sell=linreg 25 cross below linreg 50 \n" + screener_df.to_string(index=False)
    send_telegram_message(message)

# Display a sample of the screener results
print("Hourly screener Results:\n Buy=linreg 25 crossover linreg 50 \n Sell=linreg 25 cross below linreg 50 \n")
print(screener_df.tail())


In [ ]:
print(sp500_tickers)

In [ ]:
import requests_cache
session = requests_cache.CachedSession('yfinance.cache')
session.headers['User-agent'] = 'my-program/1.0'
ticker = yf.Ticker('AAPL', session=session)

# The scraped response will be stored in the cache
ticker.actions

In [ ]:
from requests import Session
from requests_cache import CacheMixin, SQLiteCache
from requests_ratelimiter import LimiterMixin, MemoryQueueBucket
from pyrate_limiter import Duration, RequestRate, Limiter
class CachedLimiterSession(CacheMixin, LimiterMixin, Session):
   pass

session = CachedLimiterSession(
   limiter=Limiter(RequestRate(2, Duration.SECOND*5)),  # max 2 requests per 5 seconds
   bucket_class=MemoryQueueBucket,
   backend=SQLiteCache("yfinance.cache"),
)

In [ ]:
import yfinance as yf
from requests import Session
from requests_cache import CacheMixin, SQLiteCache
from requests_ratelimiter import LimiterMixin, MemoryQueueBucket
from pyrate_limiter import Duration, RequestRate, Limiter
class CachedLimiterSession(CacheMixin, LimiterMixin, Session):
   pass

session = CachedLimiterSession(
   limiter=Limiter(RequestRate(2, Duration.SECOND*5)),  # max 2 requests per 5 seconds
   bucket_class=MemoryQueueBucket,
   backend=SQLiteCache("yfinance.cache"),
)
dat = yf.Tickers(sp500_tickers,session=session) 

# get historical market data
his_data = dat.history(period='1mo', interval='60m')
# Print the columns of the downloaded historical data
print(his_data.columns)


In [ ]:
import os
import time
import pytz
import requests
import pandas as pd
import numpy as np
import pandas_ta as ta
from datetime import datetime, timedelta
from bs4 import BeautifulSoup
from dotenv import load_dotenv
import yfinance as yf

# Function to check if the US market is open
def is_us_market_open():
    """Checks if the current time is within US market hours (9:30 AM - 4:00 PM ET) and not on weekends."""
    eastern = pytz.timezone('US/Eastern')
    now = datetime.now(eastern)
    if now.weekday() in [5, 6]:  # Saturday and Sunday
        return False
    market_open = now.replace(hour=9, minute=30, second=0, microsecond=0)
    market_close = now.replace(hour=16, minute=0, second=0, microsecond=0)
    return market_open <= now <= market_close

# if not is_us_market_open():
#     print("The US market is currently closed. Script execution halted.")
# else:
load_dotenv()

# Function to fetch S&P 500 tickers from Wikipedia
def get_sp500_tickers():
    url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    table = soup.find('table', {'id': 'constituents'})
    tickers = [row.find('td').text.strip() for row in table.find_all('tr')[1:]]
    return tickers

# Function to send a message via Telegram
def send_telegram_message(message):
    bot_token = os.getenv("TELEGRAM_BOT_TOKEN")
    chat_id = os.getenv("TELEGRAM_CHAT_ID")
    url = f"https://api.telegram.org/bot{bot_token}/sendMessage"
    data = {"chat_id": chat_id, "text": message}
    requests.post(url, data=data)

# Retrieve the S&P 500 tickers and save them to a CSV file
sp500_tickers = get_sp500_tickers()
os.makedirs('stockdata', exist_ok=True)
pd.DataFrame(sp500_tickers, columns=["Ticker"]).to_csv('stockdata/sp500_tickers.csv', index=False)

# Read tickers from CSV
tickers_df = pd.read_csv('stockdata/sp500_tickers.csv')
tickers = tickers_df['Ticker'].tolist()

# --- New Bulk Download Approach Using a Cached & Rate-Limited Session ---
from requests import Session
from requests_cache import CacheMixin, SQLiteCache
from requests_ratelimiter import LimiterMixin, MemoryQueueBucket
from pyrate_limiter import Duration, RequestRate, Limiter

class CachedLimiterSession(CacheMixin, LimiterMixin, Session):
    pass

session = CachedLimiterSession(
    limiter=Limiter(RequestRate(2, Duration.SECOND * 5)),  # max 2 requests per 5 seconds
    bucket_class=MemoryQueueBucket,
    backend=SQLiteCache("yfinance.cache"),
)

# Create a Tickers object with the custom session.
# yf.Tickers accepts a space-separated string of tickers.
tickers_str = " ".join(tickers)
dat = yf.Tickers(tickers_str, session=session)

# Download historical data for all tickers (1 month period, 1-day interval)
try:
    his_data = dat.history(period='1mo', interval='1d')
except Exception as e:
    print(f"Error downloading bulk data: {e}")
    his_data = None

screener_results = []

if his_data is not None:
    # Process data for each ticker individually
    for ticker in tickers:
        try:
            # Extract the ticker-specific data from the MultiIndex DataFrame
            df = his_data.xs(ticker, axis=1, level='Ticker')
            # Flatten the columns to a single level (Price types)
            df.columns = df.columns.get_level_values(0)
            
            # Round OHLC values to two decimals if available
            for col in ['Open', 'High', 'Low', 'Close']:
                if col in df.columns:
                    df[col] = df[col].round(2)

            # Calculate linear regression curves using a 25- and 50-day lookback
            df['reg1'] = ta.linreg(df['Close'], length=25)
            df['reg2'] = ta.linreg(df['Close'], length=50)

            # Generate buy and sell signals
            df['buy_signal'] = np.where((df['reg1'] > df['reg2']) & (df['reg1'].shift(1) <= df['reg2'].shift(1)), 1, 0)
            df['sell_signal'] = np.where((df['reg1'] < df['reg2']) & (df['reg1'].shift(1) >= df['reg2'].shift(1)), 1, 0)

            # Save each ticker's data to a CSV file
            csv_filename = f'stockdata/{ticker}_1d_data.csv'
            df.to_csv(csv_filename)

            # Filter for recent data (last day)
            if not df.empty:
                recent_date_cutoff = df.index.max() - pd.Timedelta(days=1)
                df_recent = df[df.index >= recent_date_cutoff]
                df_filtered = df_recent[(df_recent['buy_signal'] == 1) | (df_recent['sell_signal'] == 1)]
                if not df_filtered.empty:
                    for index, row in df_filtered.iterrows():
                        screener_results.append({
                            'Ticker': ticker,
                            'Date': index,
                            'Buy Signal': row['buy_signal'],
                            'Sell Signal': row['sell_signal']
                        })
        except Exception as e:
            print(f"Failed to process data for ticker {ticker}: {e}")

# Save the screener results to a CSV file
screener_df = pd.DataFrame(screener_results)
screener_df.to_csv('Regression_cross_screener_results_1d.csv', index=False)

# Send the results via Telegram if there are any signals
if not screener_df.empty:
    message = (
        "Daily Screener Results:\n"
        "Buy = linreg 25 crossover linreg 50\n"
        "Sell = linreg 25 cross below linreg 50\n"
        f"{screener_df.to_string(index=False)}"
    )
    # send_telegram_message(message)

# Display a sample of the screener results
print("Daily Screener Results:\nBuy = linreg 25 crossover linreg 50\nSell = linreg 25 cross below linreg 50\n")
print(screener_df.tail())


In [ ]:
import logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s %(levelname)s: %(message)s")


In [ ]:
logging.info("Retrieved %d tickers, sample: %s", len(sp500_tickers), sp500_tickers)


In [ ]:
%time sp500_tickers = get_sp500_tickers()


In [ ]:
%time
try:
    his_data = dat.history(period='1mo', interval='1d')
    print("Historical data shape:", his_data.shape)
except Exception as e:
    print("Error downloading bulk data:", e)


In [ ]:
import os
import time
import pytz
import requests
import pandas as pd
import numpy as np
import pandas_ta as ta
from datetime import datetime, timedelta
from bs4 import BeautifulSoup
from dotenv import load_dotenv
import yfinance as yf

# Check if US market is open
def is_us_market_open():
    """Checks if the current time is within US market hours (9:30 AM - 4:00 PM ET) and not on weekends."""
    eastern = pytz.timezone('US/Eastern')
    now = datetime.now(eastern)
    if now.weekday() in [5, 6]:  # Saturday and Sunday
        return False
    market_open = now.replace(hour=9, minute=30, second=0, microsecond=0)
    market_close = now.replace(hour=16, minute=0, second=0, microsecond=0)
    return market_open <= now <= market_close

# if not is_us_market_open():
#     print("The US market is currently closed. Script execution halted.")
# else:
load_dotenv()

# Function to fetch S&P 500 tickers from Wikipedia
def get_sp500_tickers():
    url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    table = soup.find('table', {'id': 'constituents'})
    tickers = [row.find('td').text.strip() for row in table.find_all('tr')[1:]]
    return tickers

# Function to send message to Telegram
def send_telegram_message(message):
    bot_token = os.getenv("TELEGRAM_BOT_TOKEN")
    chat_id = os.getenv("TELEGRAM_CHAT_ID")
    url = f"https://api.telegram.org/bot{bot_token}/sendMessage"
    data = {"chat_id": chat_id, "text": message}
    requests.post(url, data=data)

# Get the S&P 500 tickers and save to CSV
sp500_tickers = get_sp500_tickers()
os.makedirs('stockdata', exist_ok=True)
pd.DataFrame(sp500_tickers, columns=["Ticker"]).to_csv('stockdata/sp500_tickers.csv', index=False)

# Read tickers from CSV
tickers_df = pd.read_csv('stockdata/sp500_tickers.csv')
tickers = tickers_df['Ticker'].tolist()

# Define the timeframes (here using only daily data for 90 days)
timeframes = {
    '1d': timedelta(days=90),
    # Additional intervals (e.g., '15m', '1h', '1wk') can be added as needed
}
recent_period = 1  # in days

# --- New download approach using CachedLimiterSession and yf.Tickers ---
from requests import Session
from requests_cache import CacheMixin, SQLiteCache
from requests_ratelimiter import LimiterMixin, MemoryQueueBucket
from pyrate_limiter import Duration, RequestRate, Limiter

class CachedLimiterSession(CacheMixin, LimiterMixin, Session):
    pass

session = CachedLimiterSession(
    limiter=Limiter(RequestRate(2, Duration.SECOND * 5)),  # max 2 requests per 5 seconds
    bucket_class=MemoryQueueBucket,
    backend=SQLiteCache("yfinance.cache"),
)

# Create a Tickers object using all tickers with the custom session.
# Note: yf.Tickers accepts a string of tickers separated by spaces.
tickers_str = " ".join(tickers)
dat = yf.Tickers(tickers_str, session=session)

# Dictionary to hold downloaded data for each timeframe
data = {}
screener_results = []  # list to store screener signal results

# Loop over timeframes and download bulk historical data
for tf, delta in timeframes.items():
    # Use period string based on the desired date range, e.g. "90d" for 90 days
    period_str = f"{delta.days}d"
    try:
        his_data = dat.history(period=period_str, interval=tf)
    except Exception as e:
        print(f"Error downloading bulk data for interval {tf}: {e}")
        continue

    data[tf] = {}
    # his_data has a MultiIndex for columns: (Price, Ticker)
    for ticker in tickers:
        try:
            # Extract data for a single ticker from the MultiIndex DataFrame
            df = his_data.xs(ticker, axis=1, level='Ticker')
            # Flatten the columns (Price types) if needed
            df.columns = df.columns.get_level_values(0)
            # Round OHLC values to two decimals
            for col in ['Open', 'High', 'Low', 'Close']:
                if col in df.columns:
                    df[col] = df[col].round(2)

            # Calculate linear regression curves using pandas_ta
            df['reg1'] = ta.linreg(df['Close'], length=25)
            df['reg2'] = ta.linreg(df['Close'], length=50)

            # Generate buy and sell signals
            df['buy_signal'] = np.where((df['reg1'] > df['reg2']) & (df['reg1'].shift(1) <= df['reg2'].shift(1)), 1, 0)
            df['sell_signal'] = np.where((df['reg1'] < df['reg2']) & (df['reg1'].shift(1) >= df['reg2'].shift(1)), 1, 0)

            # Save each ticker's data to CSV
            csv_filename = f'stockdata/{ticker}_{tf}_data.csv'
            df.to_csv(csv_filename)

            # Filter for the recent period
            if not df.empty:
                recent_date_cutoff = df.index.max() - pd.Timedelta(days=recent_period)
                df_recent = df[df.index >= recent_date_cutoff]
                df_filtered = df_recent[(df_recent['buy_signal'] == 1) | (df_recent['sell_signal'] == 1)]
                if not df_filtered.empty:
                    for index, row in df_filtered.iterrows():
                        screener_results.append({
                            'Ticker': ticker,
                            'Date': index,
                            'Buy Signal': row['buy_signal'],
                            'Sell Signal': row['sell_signal']
                        })
        except Exception as e:
            print(f"Failed to process data for ticker {ticker} on timeframe {tf}: {e}")

# Save screener results to a CSV file
screener_df = pd.DataFrame(screener_results)
screener_df.to_csv('Regression_cross_screener_results_1d.csv', index=False)

# Send results to Telegram if available
if not screener_df.empty:
    message = (
        "Daily screener Results:\n"
        "Buy = linreg 25 crossover linreg 50\n"
        "Sell = linreg 25 cross below linreg 50\n"
        f"{screener_df.to_string(index=False)}"
    )
    # send_telegram_message(message)

# Display a sample of the screener results
print("Daily screener Results:\nBuy = linreg 25 crossover linreg 50\nSell = linreg 25 cross below linreg 50\n")
print(screener_df.tail())


# fix version 1d screener

In [ ]:
import os
import time
import pytz
import requests
import pandas as pd
import numpy as np
import pandas_ta as ta
from datetime import datetime, timedelta
from bs4 import BeautifulSoup
from dotenv import load_dotenv
import yfinance as yf

# Check if US market is open
def is_us_market_open():
    """Checks if the current time is within US market hours (9:30 AM - 4:00 PM ET) and not on weekends."""
    eastern = pytz.timezone('US/Eastern')
    now = datetime.now(eastern)
    if now.weekday() in [5, 6]:  # Saturday and Sunday
        return False
    market_open = now.replace(hour=9, minute=30, second=0, microsecond=0)
    market_close = now.replace(hour=16, minute=0, second=0, microsecond=0)
    return market_open <= now <= market_close

# For debugging in Jupyter, you can disable the market check.
# if not is_us_market_open():
#     print("The US market is currently closed. Script execution halted.")
# else:
load_dotenv()

# Function to fetch S&P 500 tickers from Wikipedia
def get_sp500_tickers():
    url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    table = soup.find('table', {'id': 'constituents'})
    tickers = [row.find('td').text.strip() for row in table.find_all('tr')[1:]]
    return tickers

# Function to send a message via Telegram
def send_telegram_message(message):
    bot_token = os.getenv("TELEGRAM_BOT_TOKEN")
    chat_id = os.getenv("TELEGRAM_CHAT_ID")
    url = f"https://api.telegram.org/bot{bot_token}/sendMessage"
    data = {"chat_id": chat_id, "text": message}
    requests.post(url, data=data)

# Get the S&P 500 tickers and save to CSV
sp500_tickers = get_sp500_tickers()
os.makedirs('stockdata', exist_ok=True)
pd.DataFrame(sp500_tickers, columns=["Ticker"]).to_csv('stockdata/sp500_tickers.csv', index=False)

# Read tickers from CSV
tickers_df = pd.read_csv('stockdata/sp500_tickers.csv')
tickers = tickers_df['Ticker'].tolist()

# Define the timeframes (here using only daily data for 90 days)
timeframes = {
    '1d': timedelta(days=90),
}
recent_period = 1  # in days

# --- New download approach using CachedLimiterSession and yf.Tickers ---
from requests import Session
from requests_cache import CacheMixin, SQLiteCache
from requests_ratelimiter import LimiterMixin, MemoryQueueBucket
from pyrate_limiter import Duration, RequestRate, Limiter

class CachedLimiterSession(CacheMixin, LimiterMixin, Session):
    pass

session = CachedLimiterSession(
    limiter=Limiter(RequestRate(2, Duration.SECOND * 5)),  # max 2 requests per 5 seconds
    bucket_class=MemoryQueueBucket,
    backend=SQLiteCache("yfinance.cache"),
)

# Create a Tickers object using all tickers with the custom session.
# Note: yf.Tickers accepts a string of tickers separated by spaces.
tickers_str = " ".join(tickers)
dat = yf.Tickers(tickers_str, session=session)

data = {}
screener_results = []  # list to store screener signal results

for tf, delta in timeframes.items():
    # Use period string based on the desired date range, e.g. "90d" for 90 days
    period_str = f"{delta.days}d"
    try:
        his_data = dat.history(period=period_str, interval=tf)
    except Exception as e:
        print(f"Error downloading bulk data for interval {tf}: {e}")
        continue

    data[tf] = {}
    # Get available tickers in the downloaded data from the MultiIndex level 'Ticker'
    available_tickers = set(his_data.columns.get_level_values('Ticker'))
    for ticker in tickers:
        if ticker not in available_tickers:
            print(f"Ticker {ticker} not found in historical data, skipping.")
            continue
        try:
            # Extract data for a single ticker from the MultiIndex DataFrame and make a copy
            df = his_data.xs(ticker, axis=1, level='Ticker').copy()
            # Flatten the columns (Price types) if needed
            df.columns = df.columns.get_level_values(0)
            # Round OHLC values to two decimals using .loc to avoid SettingWithCopyWarning
            for col in ['Open', 'High', 'Low', 'Close']:
                if col in df.columns:
                    df.loc[:, col] = df[col].round(2)

            # Calculate linear regression curves using pandas_ta
            df['reg1'] = ta.linreg(df['Close'], length=25)
            df['reg2'] = ta.linreg(df['Close'], length=50)

            # Generate buy and sell signals
            df['buy_signal'] = np.where((df['reg1'] > df['reg2']) & (df['reg1'].shift(1) <= df['reg2'].shift(1)), 1, 0)
            df['sell_signal'] = np.where((df['reg1'] < df['reg2']) & (df['reg1'].shift(1) >= df['reg2'].shift(1)), 1, 0)

            # Save each ticker's data to a CSV file
            csv_filename = f'stockdata/{ticker}_{tf}_data.csv'
            df.to_csv(csv_filename)

            # Filter for the recent period
            if not df.empty:
                recent_date_cutoff = df.index.max() - pd.Timedelta(days=recent_period)
                df_recent = df[df.index >= recent_date_cutoff]
                df_filtered = df_recent[(df_recent['buy_signal'] == 1) | (df_recent['sell_signal'] == 1)]
                if not df_filtered.empty:
                    for index, row in df_filtered.iterrows():
                        screener_results.append({
                            'Ticker': ticker,
                            'Date': index,
                            'Buy Signal': row['buy_signal'],
                            'Sell Signal': row['sell_signal']
                        })
        except Exception as e:
            print(f"Failed to process data for ticker {ticker} on timeframe {tf}: {e}")

# Save the screener results to a CSV file
screener_df = pd.DataFrame(screener_results)
screener_df.to_csv('Regression_cross_screener_results_1d.csv', index=False)

# Optionally, send results to Telegram if available
if not screener_df.empty:
    message = (
        "Daily Screener Results:\n"
        "Buy = linreg 25 crossover linreg 50\n"
        "Sell = linreg 25 cross below linreg 50\n"
        f"{screener_df.to_string(index=False)}"
    )
    # Uncomment the following line to send the message:
    send_telegram_message(message)

# Display a sample of the screener results
print("Daily Screener Results:\nBuy = linreg 25 crossover linreg 50\nSell = linreg 25 cross below linreg 50\n")
print(screener_df.tail())


# 2hr

In [2]:
import os
import pytz
import requests
import pandas as pd
import numpy as np
import pandas_ta as ta
from datetime import datetime, timedelta
from bs4 import BeautifulSoup
from dotenv import load_dotenv
import yfinance as yf

# Check if US market is open
def is_us_market_open():
    """Checks if the current time is within US market hours (9:30 AM - 4:00 PM ET) and not on weekends."""
    eastern = pytz.timezone('US/Eastern')
    now = datetime.now(eastern)
    if now.weekday() in [5, 6]:  # Saturday and Sunday
        return False
    market_open = now.replace(hour=9, minute=30, second=0, microsecond=0)
    market_close = now.replace(hour=16, minute=0, second=0, microsecond=0)
    return market_open <= now <= market_close

# For debugging in Jupyter, you can disable the market check.
# if not is_us_market_open():
#     print("The US market is currently closed. Script execution halted.")
# else:
load_dotenv()

# Function to fetch S&P 500 tickers from Wikipedia
def get_sp500_tickers():
    url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    table = soup.find('table', {'id': 'constituents'})
    tickers = [row.find('td').text.strip() for row in table.find_all('tr')[1:]]
    return tickers

# Function to send a message via Telegram
def send_telegram_message(message):
    bot_token = os.getenv("TELEGRAM_BOT_TOKEN")
    chat_id = os.getenv("TELEGRAM_CHAT_ID")
    url = f"https://api.telegram.org/bot{bot_token}/sendMessage"
    data = {"chat_id": chat_id, "text": message}
    requests.post(url, data=data)

# Get the S&P 500 tickers and save to CSV
sp500_tickers = get_sp500_tickers()
os.makedirs('stockdata', exist_ok=True)
pd.DataFrame(sp500_tickers, columns=["Ticker"]).to_csv('stockdata/sp500_tickers.csv', index=False)

# Read tickers from CSV
tickers_df = pd.read_csv('stockdata/sp500_tickers.csv')
tickers = tickers_df['Ticker'].tolist()

# Define the timeframe for downloading 60-minute data over the past 30 days.
# We will later resample these bars to 2-hour bars.
timeframes = {
    '60m': timedelta(days=30),
}
recent_period = 2  # Look back the last 2 hours for new signals

# --- New download approach using CachedLimiterSession and yf.Tickers ---
from requests import Session
from requests_cache import CacheMixin, SQLiteCache
from requests_ratelimiter import LimiterMixin, MemoryQueueBucket
from pyrate_limiter import Duration, RequestRate, Limiter

class CachedLimiterSession(CacheMixin, LimiterMixin, Session):
    pass

session = CachedLimiterSession(
    limiter=Limiter(RequestRate(2, Duration.SECOND * 5)),  # max 2 requests per 5 seconds
    bucket_class=MemoryQueueBucket,
    backend=SQLiteCache("yfinance.cache"),
)

# Create a Tickers object using all tickers with the custom session.
# Note: yf.Tickers accepts a string of tickers separated by spaces.
tickers_str = " ".join(tickers)
dat = yf.Tickers(tickers_str, session=session)

data = {}
screener_results = []  # list to store screener signal results

for tf, delta in timeframes.items():
    # Use period string based on the desired date range, e.g. "30d" for 30 days
    period_str = f"{delta.days}d"
    try:
        his_data = dat.history(period=period_str, interval=tf)
    except Exception as e:
        print(f"Error downloading bulk data for interval {tf}: {e}")
        continue

    data[tf] = {}
    # Get available tickers in the downloaded data from the MultiIndex level 'Ticker'
    available_tickers = set(his_data.columns.get_level_values('Ticker'))
    for ticker in tickers:
        if ticker not in available_tickers:
            print(f"Ticker {ticker} not found in historical data, skipping.")
            continue
        try:
            # Extract data for a single ticker from the MultiIndex DataFrame and make a copy
            df = his_data.xs(ticker, axis=1, level='Ticker').copy()
            # Flatten the columns (Price types) if needed
            df.columns = df.columns.get_level_values(0)
            # Round OHLC values to two decimals using .loc to avoid SettingWithCopyWarning
            for col in ['Open', 'High', 'Low', 'Close']:
                if col in df.columns:
                    df.loc[:, col] = df[col].round(2)

            # Resample the 60m data to 2-hour bars
            df_resampled = df.resample('2h').agg({
                'Open': 'first',
                'High': 'max',
                'Low': 'min',
                'Close': 'last',
                'Volume': 'sum'
            }).dropna()

            # Calculate linear regression curves using pandas_ta
            df_resampled['reg1'] = ta.linreg(df_resampled['Close'], length=25)
            df_resampled['reg2'] = ta.linreg(df_resampled['Close'], length=50)

            # Generate buy and sell signals
            df_resampled['buy_signal'] = np.where(
                (df_resampled['reg1'] > df_resampled['reg2']) & (df_resampled['reg1'].shift(1) <= df_resampled['reg2'].shift(1)),
                1, 0
            )
            df_resampled['sell_signal'] = np.where(
                (df_resampled['reg1'] < df_resampled['reg2']) & (df_resampled['reg1'].shift(1) >= df_resampled['reg2'].shift(1)),
                1, 0
            )

            # Save each ticker's resampled data to a CSV file
            csv_filename = f'stockdata/{ticker}_2h_data.csv'
            df_resampled.to_csv(csv_filename)

            # Filter for the recent period (last 2 hours)
            if not df_resampled.empty:
                recent_date_cutoff = df_resampled.index.max() - pd.Timedelta(hours=recent_period)
                df_recent = df_resampled[df_resampled.index >= recent_date_cutoff]
                df_filtered = df_recent[(df_recent['buy_signal'] == 1) | (df_recent['sell_signal'] == 1)]
                if not df_filtered.empty:
                    for index, row in df_filtered.iterrows():
                        screener_results.append({
                            'Ticker': ticker,
                            'Date': index,
                            'Buy Signal': row['buy_signal'],
                            'Sell Signal': row['sell_signal']
                        })
        except Exception as e:
            print(f"Failed to process data for ticker {ticker} on timeframe {tf}: {e}")

# Save the screener results to a CSV file
screener_df = pd.DataFrame(screener_results)
screener_df.to_csv('Regression_cross_screener_results_2h.csv', index=False)

# Optionally, send results to Telegram if available
if not screener_df.empty:
    message = (
        "2hr Screener Results:\n"
        "Buy = linreg 25 crossover linreg 50\n"
        "Sell = linreg 25 cross below linreg 50\n"
        f"{screener_df.to_string(index=False)}"
    )
    send_telegram_message(message)

# Display a sample of the screener results
print("2hr Screener Results:\nBuy = linreg 25 crossover linreg 50\nSell = linreg 25 cross below linreg 50\n")
print(screener_df.tail())


[**********************58%***                    ]  292 of 503 completedUnable to deserialize response: a bytes-like object is required, not 'NoneType'
[*********************100%***********************]  502 of 503 completed

3 Failed downloads:
['BF.B']: YFPricesMissingError('possibly delisted; no price data found  (period=30d)')
['TFX']: EOFError('Ran out of input')
['BRK.B']: YFPricesMissingError('possibly delisted; no price data found  (period=30d) (Yahoo error = "No data found, symbol may be delisted")')


2hr Screener Results:
Buy = linreg 25 crossover linreg 50
Sell = linreg 25 cross below linreg 50

   Ticker                      Date  Buy Signal  Sell Signal
53   TRMB 2025-02-21 20:00:00+00:00         0.0          1.0
54    WFC 2025-02-21 18:00:00+00:00         0.0          1.0
55    WST 2025-02-21 18:00:00+00:00         1.0          0.0
56    WDC 2025-02-21 18:00:00+00:00         0.0          1.0
57    ZTS 2025-02-21 20:00:00+00:00         1.0          0.0


## 2hr download new 

In [3]:
import os
import pytz
import requests
import pandas as pd
import pandas_ta as ta
import numpy as np
import yfinance as yf
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
from dotenv import load_dotenv

############################
# 1. Market Open Check
############################
def is_us_market_open():
    """
    Checks if the current time is within US market hours (9:30 AM - 4:00 PM ET)
    and not on weekends.
    """
    eastern = pytz.timezone('US/Eastern')
    now = datetime.now(eastern)
    if now.weekday() in [5, 6]:  # Saturday and Sunday
        return False
    market_open = now.replace(hour=9, minute=30, second=0, microsecond=0)
    market_close = now.replace(hour=16, minute=0, second=0, microsecond=0)
    return market_open <= now <= market_close

############################
# 2. Telegram Functions
############################
def send_telegram_message(message):
    """
    Sends a given message to a Telegram chat.
    """
    bot_token = os.getenv("TELEGRAM_BOT_TOKEN")
    chat_id = os.getenv("TELEGRAM_CHAT_ID")
    url = f"https://api.telegram.org/bot{bot_token}/sendMessage"
    data = {"chat_id": chat_id, "text": message}
    requests.post(url, data=data)

############################
# 3. Utility Functions
############################
def get_sp500_tickers():
    """
    Fetches the S&P 500 tickers from Wikipedia.
    """
    url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    table = soup.find('table', {'id': 'constituents'})
    return [row.find('td').text.strip() for row in table.find_all('tr')[1:]]

# Note: This download_data function is no longer used since we use a bulk download.
# def download_data(ticker, timeframe, start_date, end_date):
#     return yf.download(ticker, start=start_date, end=end_date, interval=timeframe)

############################
# 4. Bulk Download Setup using CachedLimiterSession and yf.Tickers
############################
from requests import Session
from requests_cache import CacheMixin, SQLiteCache
from requests_ratelimiter import LimiterMixin, MemoryQueueBucket
from pyrate_limiter import Duration, RequestRate, Limiter

class CachedLimiterSession(CacheMixin, LimiterMixin, Session):
    pass

session = CachedLimiterSession(
    limiter=Limiter(RequestRate(2, Duration.SECOND * 5)),  # max 2 requests per 5 seconds
    bucket_class=MemoryQueueBucket,
    backend=SQLiteCache("yfinance.cache"),
)

############################
# 5. Main Screener
############################
def main():
    # if not is_us_market_open():
    #     print("The US market is currently closed. Script execution halted.")
    #     return
    
    # Create directory to store data
    os.makedirs('stockdata', exist_ok=True)
    
    # Get S&P 500 tickers and save to CSV
    sp500_tickers = get_sp500_tickers()
    pd.DataFrame(sp500_tickers, columns=["Ticker"]).to_csv('stockdata/sp500_tickers.csv', index=False)
    
    # Read tickers from CSV
    tickers = pd.read_csv('stockdata/sp500_tickers.csv')['Ticker'].tolist()
    
    # Define the timeframe for bulk downloading 60m data over the past 30 days.
    # We will later resample these bars to 2-hour bars.
    timeframes = {
        '60m': timedelta(days=30),
    }
    recent_period = 2  # Look back the last 2 hours for new signals
    # Bulk download period string based on days
    # (e.g., '30d' for 30 days)
    
    # Create a Tickers object with the custom CachedLimiterSession
    tickers_str = " ".join(tickers)
    dat = yf.Tickers(tickers_str, session=session)
    
    data = {}
    screener_results = []  # list to store screener signal results
    
    # Loop over defined timeframes (in this case, only '60m')
    for tf, delta in timeframes.items():
        period_str = f"{delta.days}d"
        try:
            his_data = dat.history(period=period_str, interval=tf)
        except Exception as e:
            print(f"Error downloading bulk data for interval {tf}: {e}")
            continue
        
        data[tf] = {}
        # Extract available tickers from the MultiIndex level 'Ticker'
        available_tickers = set(his_data.columns.get_level_values('Ticker'))
        for ticker in tickers:
            if ticker not in available_tickers:
                print(f"Ticker {ticker} not found in historical data, skipping.")
                continue
            try:
                # Extract data for a single ticker and make a copy
                df = his_data.xs(ticker, axis=1, level='Ticker').copy()
                # Flatten the columns (Price types) if needed
                df.columns = df.columns.get_level_values(0)
                # Round OHLC values to two decimals to avoid floating point issues
                for col in ['Open', 'High', 'Low', 'Close']:
                    if col in df.columns:
                        df.loc[:, col] = df[col].round(2)
                
                # Resample the 60m data into 2-hour bars
                df_resampled = df.resample('2h').agg({
                    'Open': 'first',
                    'High': 'max',
                    'Low': 'min',
                    'Close': 'last',
                    'Volume': 'sum'
                }).dropna()
                
                # Calculate linear regression curves using pandas_ta
                df_resampled['reg1'] = ta.linreg(df_resampled['Close'], length=25)
                df_resampled['reg2'] = ta.linreg(df_resampled['Close'], length=50)
                
                # Generate buy and sell signals
                df_resampled['buy_signal'] = np.where(
                    (df_resampled['reg1'] > df_resampled['reg2']) & (df_resampled['reg1'].shift(1) <= df_resampled['reg2'].shift(1)),
                    1, 0
                )
                df_resampled['sell_signal'] = np.where(
                    (df_resampled['reg1'] < df_resampled['reg2']) & (df_resampled['reg1'].shift(1) >= df_resampled['reg2'].shift(1)),
                    1, 0
                )
                
                # Save the resampled data to a CSV file for this ticker
                csv_filename = f'stockdata/{ticker}_2h_data.csv'
                df_resampled.to_csv(csv_filename)
                
                # Filter for the recent period (last 2 hours)
                if not df_resampled.empty:
                    recent_date_cutoff = df_resampled.index.max() - pd.Timedelta(hours=recent_period)
                    df_recent = df_resampled[df_resampled.index >= recent_date_cutoff]
                    df_filtered = df_recent[(df_recent['buy_signal'] == 1) | (df_recent['sell_signal'] == 1)]
                    if not df_filtered.empty:
                        for index, row in df_filtered.iterrows():
                            screener_results.append({
                                'Ticker': ticker,
                                'Date': index,
                                'Buy Signal': row['buy_signal'],
                                'Sell Signal': row['sell_signal']
                            })
            except Exception as e:
                print(f"Failed to process data for ticker {ticker} on timeframe {tf}: {e}")
    
    # Save the screener results to a CSV file
    screener_df = pd.DataFrame(screener_results)
    screener_df.to_csv('Regression_cross_screener_results_2h.csv', index=False)
    
    # If signals exist, print a sample and send them via Telegram
    if not screener_df.empty:
        print(f"2hr Screener Results:\n{str(screener_df.tail())}")
        message = (
            "2hr Screener Results:\n"
            "Buy = linreg(25) crosses above linreg(50)\n"
            "Sell = linreg(25) crosses below linreg(50)\n\n"
            + screener_df.to_string(index=False)
        )
        send_telegram_message(message)
    else:
        print("No buy/sell signals found in the last 2 hours.")

############################
# 6. Entry Point
############################
if __name__ == "__main__":
    main()


[**********************60%****                   ]  304 of 503 completedUnable to deserialize response: a bytes-like object is required, not 'NoneType'
[*********************100%***********************]  503 of 503 completed

3 Failed downloads:
['MKTX']: InterfaceError('bad parameter or other API misuse')
['BF.B']: YFPricesMissingError('possibly delisted; no price data found  (period=30d)')
['BRK.B']: YFPricesMissingError('possibly delisted; no price data found  (period=30d) (Yahoo error = "No data found, symbol may be delisted")')


2hr Screener Results:
   Ticker                      Date  Buy Signal  Sell Signal
53   TRMB 2025-02-21 20:00:00+00:00         0.0          1.0
54    WFC 2025-02-21 18:00:00+00:00         0.0          1.0
55    WST 2025-02-21 18:00:00+00:00         1.0          0.0
56    WDC 2025-02-21 18:00:00+00:00         0.0          1.0
57    ZTS 2025-02-21 20:00:00+00:00         1.0          0.0
